In [ ]:
import os
import time
import pickle
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

COOKIES_FILE = "cookies.pkl"


def manual_login_and_save_cookies():
    """Abre navegador para login manual y guarda cookies."""
    driver = webdriver.Chrome()
    driver.get("https://autos.mercadolibre.cl/")

    print("👉 Inicia sesión manualmente en el navegador abierto...")
    print("⏳ Tienes 60 segundos para hacerlo")

    # Te da 60 segundos para completar login
    time.sleep(120)

    # Guardar cookies
    pickle.dump(driver.get_cookies(), open(COOKIES_FILE, "wb"))
    driver.quit()
    print("✅ Cookies guardadas en", COOKIES_FILE)


def load_session_from_cookies():
    """Carga cookies en Requests."""
    cookies = pickle.load(open(COOKIES_FILE, "rb"))
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    return session


def scrape_autos(session):
    """Ejemplo de scraping: lista títulos de autos"""
    url = "https://autos.mercadolibre.cl/"
    response = session.get(url)

    if response.status_code != 200:
        print("⚠️ Error al acceder:", response.status_code)
        return

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    titulos = [item.get_text(strip=True) for item in soup.select("h2.ui-search-item__title")]

    print("\n🚗 Autos encontrados:")
    for t in titulos:
        print("-", t)


if __name__ == "__main__":
    if not os.path.exists(COOKIES_FILE):
        manual_login_and_save_cookies()

    session = load_session_from_cookies()
    scrape_autos(session)


In [2]:
import pickle

cookies = pickle.load(open("cookies.pkl", "rb"))
print(len(cookies), "cookies guardadas")
for c in cookies[:5]:  # mostrar las 5 primeras
    print(c["name"], "=", c["value"])


24 cookies guardadas
_snoopy = eyJmaW5nZXJwcmludCI6IlNNQWhMcHdESTY2NUpjb09tZ3pldDJ2RTI0U3hlYlRlY29lSkEzMkJTQWwvdFQ4eUdQbG1keHc4aVNqNjBuaDBFdE9LK2taUkNESWtCTC9xTGpBc0RYTU9hOFdqNVNyWHowMzNqYVgvbi9ZdDdJUjVWV3RabzZQTXUzTW9QTFFuek16bEZUQTd4SXR4dDJyd0FSWEI3OVlqUmRNd3BienNVRkN0WXEzZlE4NFRXR3M5QVc0PSIsImtleSI6InB1QlRqUFhib2VNMGwwL1A3cDhSdVJqYTRmZkZMQmd4RnM5cytxUWdLQnZwZXhRSXhsdFJ1L1JmanlhTEpOMC8zbnQzT2ZQOUpLbVhpL2pOSXU2WjBxcGQwbXh3S0JvQyJ9
_mldataSessionId = 8250479a-f7b8-4171-8abf-96bcbef3a0eb
c_ui-navigation = 6.6.146
last_query = 
cp = CL-RM_TUxDQ1BST2NhYjU3


In [3]:
import requests
import pickle



# Cargar cookies
cookies = pickle.load(open("cookies.pkl", "rb"))

# Pasarlas a requests.Session
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

# Agregar headers de navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

# Probar acceso
url = "https://autos.mercadolibre.cl/"
response = session.get(url, headers=headers)

print("Status:", response.status_code)
print("URL final:", response.url)
print("HTML parcial:", response.text[:500])


Status: 200
URL final: https://autos.mercadolibre.cl/
HTML parcial: 
<!DOCTYPE html>
<html lang="es-CL">
<head><link rel="preconnect" href="https://www.google-analytics.com"/><link rel="preconnect" href="https://www.google.com"/><link rel="preconnect" href="https://data.mercadolibre.com"/><link rel="preconnect" href="https://http2.mlstatic.com"/><link rel="preconnect" href="https://stats.g.doubleclick.net"/><link rel="preconnect" href="https://analytics.mercadolibre.cl"/><link rel="preconnect" href="https://analytics.mercadolibre.com"/><link rel="preconnect" hre


In [4]:
import requests
import pickle
from bs4 import BeautifulSoup
import pandas as pd

# Cargar cookies guardadas
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

# Headers para simular navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

# URL de prueba
url = "https://autos.mercadolibre.cl/"
response = session.get(url, headers=headers)

soup = BeautifulSoup(response.text, "html.parser")

# Extraer cantidad total de resultados
total_resultados = soup.select_one("span.ui-search-search-result__quantity-results")
total_resultados_text = total_resultados.get_text(strip=True) if total_resultados else "N/A"

print("Total resultados:", total_resultados_text)

# Extraer categorías de carrocería
categorias = []
filtros = soup.select("div.ui-search-filter-dl li.ui-search-filter-container a.ui-search-link")


OUTPUT_CSV = "vehiculos_totales.csv"


for f in filtros:
    nombre = f.select_one("span.ui-search-filter-name").get_text(strip=True)
    cantidad = f.select_one("span.ui-search-filter-results-qty").get_text(strip=True)
    categorias.append({"categoria": nombre, "resultados": cantidad})

# Crear DataFrame y guardar en CSV
df = pd.DataFrame(categorias)
df.to_csv("categorias_autos.csv", index=False, encoding="utf-8-sig")

print("CSV creado con las categorías:", df)


Total resultados: 20.626 resultados
CSV creado con las categorías:                    categoria resultados
0     Solo tiendas oficiales    (7.700)
1         RM (Metropolitana)   (14.017)
2                 Valparaíso    (1.607)
3                     Biobío      (975)
4               La Araucanía      (749)
..                       ...        ...
93    Con aire acondicionado    (6.972)
94                Con alarma    (5.187)
95  Con cristales eléctricos    (6.159)
96     Vehiculos verificados    (1.752)
97            Publicados hoy      (798)

[98 rows x 2 columns]


In [5]:
import requests
import pickle
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import os
import re


OUTPUT_CSV = "vehiculos_totales.csv"
URLML = "https://autos.mercadolibre.cl/"

# Cargar cookies
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

# Hacer request
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer total de resultados
total_resultados = soup.select_one("span.ui-search-search-result__quantity-results")
if total_resultados:
    total_vehiculos = total_resultados.get_text(strip=True)
    # Quitar puntos y convertir a int
    total_vehiculos = int(re.sub(r"[^\d]", "", total_vehiculos))
else:
    total_vehiculos = 0

# Timestamp
now = datetime.now().isoformat()

# Guardar en CSV
file_exists = os.path.isfile(OUTPUT_CSV)
with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
    fieldnames = ["timestamp", "URLML", "total_vehiculos"]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()
    writer.writerow({
        "timestamp": now,
        "URLML": URLML,
        "total_vehiculos": total_vehiculos
    })

print(f"✅ Información guardada en {OUTPUT_CSV}: {total_vehiculos} vehículos")


✅ Información guardada en vehiculos_totales.csv: 20626 vehículos


In [6]:
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import re

# --- Configuración ---

CSV_EXISTENTE = "yapo_filters.csv"
OUTPUT_CSV = "yapo_filters.csv"
URLML = "https://autos.mercadolibre.cl/"

# Cargar cookies
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

# Hacer request a la página de autos
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer filtros de tipo de carrocería
filtros_ml_data = {}
filtros = soup.select("ul li.ui-search-filter-container a.ui-search-link")
for f in filtros:
    nombre = f.select_one("span.ui-search-filter-name").get_text(strip=True)
    total_text = f.select_one("span.ui-search-filter-results-qty").get_text(strip=True)
    total = int(re.sub(r"[^\d]", "", total_text))
    url = f['href']
    filtros_ml_data[nombre] = {"url": url, "total": total}

print("Filtros extraídos de ML:")
for k, v in filtros_ml_data.items():
    print(k, v["total"], v["url"])

# --- Cargar CSV existente ---
df = pd.read_csv(CSV_EXISTENTE)

# Mapear categorías a los filtros de ML
categorias_map = {
    "Autos Usados": ["SUV", "Hatchback", "Sedán", "Pick-Up", "Van", "Furgón", "Light Truck", "Coupé", "Minivan"],
    "Camiones y Buses": ["Pick-Up", "Light Truck", "Furgón"],
    "Motos": [],  # Si quisieras, puedes mapear a motos
    "Maquinaria pesada": [],
    "Arriendo de Autos": [],
    "Yo busco": []
}

# Rellenar columna con filtros de ML + URL + total
def map_filtros(categoria):
    filtros = categorias_map.get(categoria, [])
    return ", ".join([f"{f} ({filtros_ml_data[f]['total']})" for f in filtros if f in filtros_ml_data])

df["filtros_ML"] = df["label_text"].apply(map_filtros)

# Guardar CSV final
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ CSV actualizado guardado en {OUTPUT_CSV}")


Filtros extraídos de ML:
Solo tiendas oficiales 7700 https://autos.mercadolibre.cl/_Tienda_all#applied_filter_id%3Dofficial_store%26applied_filter_name%3DTiendas+oficiales%26applied_filter_order%3D2%26applied_value_id%3Dall%26applied_value_name%3DSolo+tiendas+oficiales%26applied_value_order%3D1%26applied_value_results%3D7700%26is_custom%3Dfalse
RM (Metropolitana) 14017 https://autos.mercadolibre.cl/rm-metropolitana/#applied_filter_id%3Dstate%26applied_filter_name%3DUbicaci%C3%B3n%26applied_filter_order%3D3%26applied_value_id%3DTUxDUE1FVEExM2JlYg%26applied_value_name%3DRM+%28Metropolitana%29%26applied_value_order%3D13%26applied_value_results%3D14017%26is_custom%3Dfalse
Valparaíso 1607 https://autos.mercadolibre.cl/valparaiso/#applied_filter_id%3Dstate%26applied_filter_name%3DUbicaci%C3%B3n%26applied_filter_order%3D3%26applied_value_id%3DTUxDUFZBTE84MDVj%26applied_value_name%3DValpara%C3%ADso%26applied_value_order%3D15%26applied_value_results%3D1607%26is_custom%3Dfalse
Biobío 975 https:/

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

CSV_EXISTENTE = "yapo_filters.csv"
OUTPUT_CSV = "yapo_filters_actualizado.csv"
URLML = "https://autos.mercadolibre.cl/"

# Request a ML autos
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}
response = requests.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# 1. Extraer filtros de TIPO DE VEHÍCULO de ML 🚗
filtros_ml_data = {}
tipo_vehiculo_div = None
filter_containers = soup.find_all("div", class_="ui-search-filter-groups")
for container in filter_containers:
    if container.find("h3", string="Tipo de vehículo"):
        tipo_vehiculo_div = container
        break

if tipo_vehiculo_div:
    for f in tipo_vehiculo_div.select("li a.ui-search-link"):
        nombre = f.select_one("span.ui-search-filter-name").get_text(strip=True)
        url = f['href']
        filtros_ml_data[nombre] = url
else:
    print("⚠️ 'Tipo de vehículo' filter section not found on the page.")

print("Filtros de tipo de vehículo extraídos:", filtros_ml_data)

# Cargar CSV existente
try:
    df = pd.read_csv(CSV_EXISTENTE)
except FileNotFoundError:
    print(f"⚠️ El archivo {CSV_EXISTENTE} no se encontró, se creará uno nuevo.")
    df = pd.DataFrame(columns=["label_text"])

# Mapeo categorías -> filtros ML
categorias_map = {
    "Autos Usados": ["SUV", "Hatchback", "Sedán", "Pick-Up", "Van", "Furgón", "Light Truck", "Coupé", "Minivan"],
    "Camiones y Buses": ["Pick-Up", "Light Truck", "Furgón"],
    "Motos": ["Motocicleta"],
    "Maquinaria pesada": ["Maquinaria pesada"],
    "Arriendo de Autos": [],
    "Yo busco": []
}

# Función robusta para obtener filtros y URLs
def map_filtros(categoria):
    filtros = categorias_map.get(categoria, [])
    nombres = []
    urls = []
    for f in filtros:
        nombres.append(f)
        urls.append(filtros_ml_data.get(f, ""))  # si no existe, ponemos string vacío
    return "; ".join(nombres), "; ".join(urls)

# Agregar columnas name y url_ML sin fallar aunque algún filtro esté vacío
if "label_text" in df.columns:
    results = df["label_text"].apply(map_filtros)
    df["name"] = results.apply(lambda x: x[0])
    df["url_ML"] = results.apply(lambda x: x[1])
else:
    print("⚠️ La columna 'label_text' no existe en el CSV. Se agregarán columnas vacías.")
    df["name"] = ""
    df["url_ML"] = ""

# Guardar CSV
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ CSV actualizado guardado en {OUTPUT_CSV}")


⚠️ 'Tipo de vehículo' filter section not found on the page.
Filtros de tipo de vehículo extraídos: {}
✅ CSV actualizado guardado en yapo_filters_actualizado.csv


In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import re
from datetime import datetime

# --- Configuración ---
CSV_EXISTENTE = "competencia.csv"
URLML = "https://autos.mercadolibre.cl/"

# Mapeo de categorías válidas
categorias_map = {
    "Autos Usados": [
        "SUV", "Hatchback", "Sedán", "Pick-Up", "Van", "Furgón",
        "Light Truck", "Coupé", "Minivan", "Cabriolet", "Crossover",
        "Monovolumen", "Off-Road", "Roadster", "Rural", "Eléctrico", "Híbrido", "4x4"
    ],
    "Camiones y Buses": ["Furgón", "Light Truck", "Pick-Up"],
    "Motos": ["Motocicleta", "Scooters eléctricos"],
    "Maquinaria pesada": ["Maquinaria pesada"],
    "Autos Nuevos": [],
    "Arriendo de Autos": [],
    "Yo busco": []
}

# --- Categorías que no aparecen en el filtro ---
extra_urls = {
    "Crossover": "https://autos.mercadolibre.cl/crossover/",
    "Monovolumen": "https://autos.mercadolibre.cl/monovolumen/",
    "Rural": "https://autos.mercadolibre.cl/rural/",
    "Off-Road": "https://autos.mercadolibre.cl/off-road/",
    "Roadster": "https://autos.mercadolibre.cl/roadster/",
    "Scooters eléctricos": "https://motos.mercadolibre.cl/scooters-electricos",
    "Motocicleta": "https://motos.mercadolibre.cl/motos",
    "Eléctrico": "https://autos.mercadolibre.cl/electrico/",
    "Híbrido": "https://autos.mercadolibre.cl/hibrido",
    "4x4": "https://autos.mercadolibre.cl/4x4",
}

# --- Conexión con cookies ---
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

categorias = []

# --- Scraping principal ---
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extraer total de vehículos
total_resultados = soup.select_one("span.ui-search-search-result__quantity-results")
if total_resultados:
    total_vehiculos = int(re.sub(r"[^\d]", "", total_resultados.get_text(strip=True)))
else:
    total_vehiculos = 0

# Categorías de la página
for a in soup.select("div.ui-search-filter-dl ul li a.ui-search-link"):
    label_tag = a.select_one("span.ui-search-filter-name")
    count_tag = a.select_one("span.ui-search-filter-results-qty")

    if not label_tag or not count_tag:
        continue

    label = label_tag.get_text(strip=True)
    count_text = count_tag.get_text(strip=True)
    count = int(re.sub(r"[^\d]", "", count_text))
    url = a.get("href")

    if any(label in subs for subs in categorias_map.values()):
        grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")
        
        categorias.append({
            "timestamp": datetime.now().isoformat(),
            "categoria": grupo,
            "competencia": "Mercadolibre",
            "cantidad": count,
            })

# --- Scraping de categorías extra ---
for label, url in extra_urls.items():
    resp = session.get(url, headers=headers)
    extra_soup = BeautifulSoup(resp.text, "html.parser")
    count_tag = extra_soup.select_one("span.ui-search-search-result__quantity-results")
    
    if not count_tag:
        continue
    
    count = int(re.sub(r"[^\d]", "", count_tag.get_text(strip=True)))
    grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")

    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "categoria": grupo,
        "competencia": "Mercadolibre",
        "cantidad": count,
    })


# --- Ajustar totales ---
suma_categorias = sum(item["cantidad"] for item in categorias)
diferencia = total_vehiculos - suma_categorias

if diferencia != 0:
    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "categoria": "Autos sin categoría",
        "cantidad": diferencia,
        "url": "Mercadolibre",
    })

# --- Guardar en CSV ---
df = pd.DataFrame(categorias)

try:
    old_df = pd.read_csv(CSV_EXISTENTE)
    df_final = pd.concat([old_df, df], ignore_index=True)
except FileNotFoundError:
    df_final = df

df_final.to_csv(CSV_EXISTENTE, index=False, encoding="utf-8-sig")

print("✅ Datos guardados en", CSV_EXISTENTE)
print(f"🔎 Total MercadoLibre: {total_vehiculos}")
print(f"📊 Suma categorías: {suma_categorias}")
print(f"➕ Diferencia ajustada en 'Autos sin categoría': {diferencia}")


✅ Datos guardados en competencia.csv
🔎 Total MercadoLibre: 20626
📊 Suma categorías: 22563
➕ Diferencia ajustada en 'Autos sin categoría': -1937


In [9]:
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import re
from datetime import datetime

# --- Configuración ---
CSV_EXISTENTE = "yapo_filters.csv"
URLML = "https://autos.mercadolibre.cl/"

# Mapeo de categorías
categorias_map = {
    "Autos Usados": [
        "SUV", "Hatchback", "Sedán", "Pick-Up", "Van", "Furgón",
        "Light Truck", "Coupé", "Minivan", "Cabriolet", "Crossover",
        "Monovolumen", "Off-Road", "Roadster", "Rural"
    ],
    "Camiones y Buses": ["Furgón", "Light Truck", "Pick-Up"],
    "Motos": ["Motocicleta"],
    "Maquinaria pesada": ["Maquinaria pesada"],
    "Autos Nuevos": [],
    "Arriendo de Autos": [],
    "Yo busco": []
}

# --- Categorías que no aparecen en el filtro ---
extra_urls = {
    "Crossover": "https://autos.mercadolibre.cl/crossover/",
    "Monovolumen": "https://autos.mercadolibre.cl/monovolumen/",
    "Rural": "https://autos.mercadolibre.cl/rural/",
    "Off-Road": "https://autos.mercadolibre.cl/off-road/",
    "Roadster": "https://autos.mercadolibre.cl/roadster/",
}

# --- Conexión con cookies ---
cookies = pickle.load(open("cookies.pkl", "rb"))
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "es-CL,es;q=0.9,en;q=0.8"
}

categorias = []

# --- Scraping de las categorías principales ---
response = session.get(URLML, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

for a in soup.select("div.ui-search-filter-dl ul li a.ui-search-link"):
    label_tag = a.select_one("span.ui-search-filter-name")
    count_tag = a.select_one("span.ui-search-filter-results-qty")

    if not label_tag or not count_tag:
        continue

    label = label_tag.get_text(strip=True)
    count_text = count_tag.get_text(strip=True)
    count = int(re.sub(r"[^\d]", "", count_text))
    url = a.get("href")

    # Asignar grupo
    grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")

    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "label_text": label,
        "name": grupo,
        "count": count,
        "url": url
    })

# --- Scraping de categorías extra ---
for label, url in extra_urls.items():
    resp = session.get(url, headers=headers)
    extra_soup = BeautifulSoup(resp.text, "html.parser")
    count_tag = extra_soup.select_one("span.ui-search-search-result__quantity-results")
    
    if not count_tag:
        continue
    
    count_text = count_tag.get_text(strip=True)
    count = int(re.sub(r"[^\d]", "", count_text))

    grupo = next((cat for cat, subs in categorias_map.items() if label in subs), "Otros")

    categorias.append({
        "timestamp": datetime.now().isoformat(),
        "label_text": label,
        "name": grupo,
        "count": count,
        "url": url
    })

# --- Guardar en CSV ---
df = pd.DataFrame(categorias)

try:
    old_df = pd.read_csv(CSV_EXISTENTE)
    df_final = pd.concat([old_df, df], ignore_index=True)
except FileNotFoundError:
    df_final = df

df_final.to_csv(CSV_EXISTENTE, index=False, encoding="utf-8-sig")

print("✅ Datos guardados en", CSV_EXISTENTE)


✅ Datos guardados en yapo_filters.csv
